# 6. 데이터 로딩, 저장, 파일형식

일반적으로 입/출력은 

- 텍스트 파일을 이용하는 방법
- 데이터베이스를 이용하는 방법
- 웹 API를 이용해서 네트워크를 통해 불러오는 방법

## 6.1 텍스트 파일 이용하는 방법

pandas는 표 형식의 자료로 된 텍스트데이터를 DataFrame객체로 읽어오는 몇 가지 기능을 제공한다. 

pandas 파일 파싱 함수
- read_csv: 파일, URL 또는 파일과 유사한 객체로부토 구분된 데이터를 읽어온다. 데이터 구분자(delimiter)는 쉼표(,)를 기본으로 한다. 
- read_table: 파일, URL 또는 파일과 유사한 객체로부터 구분된 데이터를 읽어온다. 데이터 구분자는 '\t'을 기본으로 한다. 
- read_fwf: 고정폭 칼럼 형식에서 데이터를 읽어온다. delimiter가 없는 데이터
- read_clipboard: 클립보드에 있는 데이터를 읽어오는 read_table 함수. 웹페이지에서 표를 긁어올 때 유용하다. 

옵션
- index: 반환하는 DataFrame에서 하나 이상의 칼럼을 index로 지정할 수 있다. 파일이나 사용자로부터 칼럼 이름을 받거나 아무것도 받지 않을 수 있다. 
- 자료형 추론과 데이터 변환: 사용자가 정의 값 변환과 비어있는 값을 위한 사용자 리스트를 포함한다. 
- 날짜 분석: 여러 칼럼에 있는 날짜와 시간정보를 하나의 칼럼에 조합해서 결과에 반영한다. 
- 반복: 여러 파일에 걸쳐 있는 자료를 반복적으로 읽어올 수 있다. 
- 정제되지 않은 데이터 처리: 행이나 꼬리말, 주석 건너뛰기 또는 천 단위마다 쉼표로 구분된 숫자 등의 데이터를 처리해 준다. 

자료형 추론가능은 어떤 칼럼이 숫자인지 불리언인지 혹은 문자열인지를 추론한다

In [1]:
!cat ./examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

이 파일은 쉼표로 구분되어 있기 때문에 read_csv를 사용해 DataFrame으로 읽어올 수 있다. 

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('./examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_table('./examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


칼럼의 이름이 없는 파일을 읽으면...

header옵션을 None으로 해준다. name옵션을 이용해 칼럼이름을 지정해 줄 수있다. 

In [6]:
!cat ./examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [7]:
pd.read_csv('./examples/ex2.csv')

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [8]:
pd.read_csv('./examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv('./examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


message 칼럼을 index으로 하는 DataFrame을 반환하려면 index_col 인자에 네 번째 또는 'message'라는 칼럼을 지정해서 index으로 만든다. 

In [10]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


계층적 index를 지정하고 싶으면 칼럼 번호나 이름의 리스트를 넘긴다. 

In [11]:
!cat ./examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [12]:
parsed = pd.read_csv('./examples/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

고정된 delimiter없이 공백이나 다른 패턴으로 필드를 구분해 놓은 경우는 read_table의 delimiter로 정규표현식을 사용한다.  

In [13]:
list(open('./examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [14]:
result = pd.read_table('./examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491
